In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  8 09:50:11 2021

@author: anwar
"""

from flask import request
from flask import Flask, jsonify
import json
import requests
import subprocess
from launchAlgo import transform
from flask_restful import Api
from flask_cors import CORS
app = Flask(__name__)
api = Api(app)
cors = CORS(app, resources={r"/*": {"origins": "*"}})
from instance.Instance import Instance
from Problem.Instance_from_Json import createInstance

instance=Instance()
Instance=createInstance(instance)
url=''


@app.route('/getjson/', methods=['GET'])
def getjson():
    f = open(r"instanceExamples/data.json")

    data = json.load(f)
    response = app.response_class(
        response=json.dumps(data),
        mimetype='application/json'
    )
    return response

@app.route('/getenergy/', methods=['GET'])
def getcsv():
    f = open(r"./energy.json")

    data = json.load(f)
    response = app.response_class(
        response=json.dumps(data),
        mimetype='application/json'
    )
    return response


# @app.route('/geturl/', methods=['GET'])
# def geturl():
    

    
#     response = app.response_class(
#         response=json.dumps(url),
#         mimetype='application/json'
#     )
#     return response

@app.route('/geturl/', methods=['POST'])
def geturl():
    
    content = request.get_data()
    c=json.loads(content)
    
    print('content',str(c))
    global url
    url=str(c)
    
    return 'url posted'


@app.route('/update/', methods=['POST'])

def events():
    
    content = request.get_data()
    c=json.loads(content)
    #print(c)
    with open(r"instanceExamples/data.json", 'w') as f:
        json.dump(c, f)
    return 'JSON posted'


@app.route('/getStatus/', methods=['POST'])
def getStatus():


    content = request.get_data()
    c=json.loads(content)
    print(c)
    update(c)
    return json.dumps(c)
    
    # with open(r"./instanceExamples/data.json", "r") as file:

    #     data= json.load(file)
    #     data['objectives']=c

    # with open(r"./instanceExamples/data.json", "w") as file:
    #     json.dump(data, file)
    # return json.dumps(c)
    
    
   
def update(result):
    with open(r"./instanceExamples/data.json", "r") as file:
        data= json.load(file)
    for row in result:
        for node in data['nodes']:
            if(node['name']==row[0]):
                print(node['name'])
                print(row[2])
                node['Manager Status']=row[2]
                if(row[2]=='Reachable'):
                    
                    #cmd = ('docker-machine ssh manager docker node promote '+str(node['name'])).split()
                    cmd = ('docker-machine ssh '+str(Instance.nodes[0].name)+' docker node promote '+str(node['name'])).split()
                    #cmd = ('ssh root@'+str(Instance.nodes[0].name)+' docker node promote '+str(node['name'])).split()
                    
                    print(cmd)
                    p = subprocess.Popen(cmd)
                    output, errors = p.communicate()
                    
    with open(r"./instanceExamples/data.json", "w") as file:
        json.dump(data, file)
        
    

@app.route('/getweights/', methods=['POST'])
def weights():


    content = request.get_data()
    c=json.loads(content)
    #print(c)
    with open(r"./instanceExamples/data.json", "r") as file:

        data= json.load(file)
        data['objectives']=c

    with open(r"./instanceExamples/data.json", "w") as file:
        json.dump(data, file)
    return json.dumps(c)

@app.route('/UpdateStatus/', methods=['GET'])
def get_nodes():

        machines=[]

        with  open(r"./test3.txt",'w') as file :


            #cmd = ('docker-machine ssh manager docker node ls').split()
            cmd = ('docker-machine ssh '+str(Instance.nodes[0].name)+ ' docker node ls').split()
            #cmd = ('ssh root@'+str(Instance.nodes[0].name)+ ' docker node ls').split()

            p = subprocess.Popen(cmd,stdout=file)
            output, errors = p.communicate()
            
        info=[]

        with open(r"./test3.txt",'r') as file:

            
            for line in file:
                
                line=line.replace("*",'')
                groupe=line.split()
                info.append(groupe)
                
                machines.append(groupe[1])

            del machines[0]
            print(info)
            if (info[1][4]=='Unreachable'):
               
                msg="Leader is down, another manager has been selected!"
                with open(r"./instanceExamples/data.json", "r") as file:
                    data= json.load(file)
                    data['nodes'][0]['activated']=="false"
                
               
                return(jsonify(msg))
            else:
                #print(groupe)
                msg=""
                #print(msg)
                return(jsonify(msg))



@app.route('/default/', methods=['GET'])
def get():
    
    cmd = ('docker-machine '+str(Instance.nodes[0].name)+' docker stack deploy --compose-file initial-docker-compose.yml p').split()
    #cmd = ('ssh root@'+str(Instance.nodes[0].name)+' docker stack deploy --compose-file initial-docker-compose.yml p').split()

    p = subprocess.Popen(cmd,stdout = subprocess.PIPE)
    output, errors = p.communicate()

    print(output)
    print(errors)
    result=jsonify("done")

    return (result)

# @app.route('/getcpu/', methods=['GET'])
# def get_cpu_per_container():
#     Instance=createInstance(instance)
#     for node in Instance.nodes:

#         r = requests.get(url+'api/v1/query?query=sum(irate(container_cpu_usage_seconds_total%7Bcontainer_label_com_docker_swarm_node_id%3D~"'+str(node.cluster_id[0])+'"%2C%20id%3D~"%2Fdocker%2F.*"%7D%5B5m%5D))%20by%20(name)%20*%20100%20&g0.tab=1')

#         for metric in json.loads(r.text)['data']['result']:

#             name=(metric['metric']['name'])

#             with open(r"./instanceExamples/data.json", "r") as file:
#                 data= json.load(file)
#             for con in data['containers']:
#                 if (con['name'] in name):

#                     con['cpu_usage']=float(metric['value'][1])

#             with open(r"./instanceExamples/data.json", "w") as file:

#                 json.dump(data, file)



#     return (jsonify('done'))

@app.route('/getmem/', methods=['GET'])
def get_mem_per_container():
    Instance=createInstance(instance)
    for node in Instance.nodes:

        r = requests.get(url+'api/v1/query?query=avg_over_time(container_memory_usage_bytes%7Bcontainer_label_com_docker_swarm_node_id%3D~"'+str(node.cluster_id[0])+'"%2C%20id%3D~"%2Fdocker%2F.*"%7D%5B5m%5D)%2F1024%2F1024&g0.tab=1')


        for metric in json.loads(r.text)['data']['result']:

            name=(metric['metric']['name'])
            with open(r"./instanceExamples/data.json", "r") as file:
                data= json.load(file)
            for con in data['containers']:
                if (con['name'] in name):

                    con['mem_usage']=float(metric['value'][1])

            with open(r"./instanceExamples/data.json", "w") as file:

                json.dump(data, file)

    for node in Instance.nodes:
        #print(node.cluster_id)
        r = requests.get(url+'api/v1/query?query=sum(node_memory_MemTotal_bytes%20*%20on(instance)%20group_left(node_name)%20node_meta%7Bnode_id%3D~"'+str(node.cluster_id[0])+'"%7D)%2F1000%2F1000&g0.tab=1')

        #print(json.loads(r.text)['data']['result'][0]['value'][1])

        with open(r"./instanceExamples/data.json", "r") as file:
            data= json.load(file)
        for con in data['nodes']:

            if (con['name'] == node.name):


                con['Maxmem']=float(json.loads(r.text)['data']['result'][0]['value'][1])

        with open(r"./instanceExamples/data.json", "w") as file:

            json.dump(data, file)
            
    for node in Instance.nodes:

        r = requests.get(url+'api/v1/query?query=sum(irate(container_cpu_usage_seconds_total%7Bcontainer_label_com_docker_swarm_node_id%3D~"'+str(node.cluster_id[0])+'"%2C%20id%3D~"%2Fdocker%2F.*"%7D%5B5m%5D))%20by%20(name)%20*%20100%20&g0.tab=1')

        for metric in json.loads(r.text)['data']['result']:

            name=(metric['metric']['name'])

            with open(r"./instanceExamples/data.json", "r") as file:
                data= json.load(file)
            for con in data['containers']:
                if (con['name'] in name):

                    con['cpu_usage']=float(metric['value'][1])

            with open(r"./instanceExamples/data.json", "w") as file:

                json.dump(data, file)

    return (jsonify('done'))


# @app.route('/getMaxmem/', methods=['GET'])
# def get_Maxmem_penode():
#     Instance=createInstance(instance)
#     for node in Instance.nodes:
#         #print(node.cluster_id)
#         r = requests.get(url+'api/v1/query?query=sum(node_memory_MemTotal_bytes%20*%20on(instance)%20group_left(node_name)%20node_meta%7Bnode_id%3D~"'+str(node.cluster_id[0])+'"%7D)%2F1000%2F1000&g0.tab=1')

#         #print(json.loads(r.text)['data']['result'][0]['value'][1])

#         with open(r"./instanceExamples/data.json", "r") as file:
#             data= json.load(file)
#         for con in data['nodes']:

#             if (con['name'] == node.name):


#                 con['Maxmem']=float(json.loads(r.text)['data']['result'][0]['value'][1])

#         with open(r"./instanceExamples/data.json", "w") as file:

#             json.dump(data, file)



#     return (jsonify('done'))



@app.route('/newapproach/', methods=['GET'])

def new_approach():

    
    
    time=transform(Instance)
    y = {"containers":len(Instance.containers),
     "exectime": round(time, 2)

    }
    with open(r"./instanceExamples/exec.json", "r") as file:
        data= json.load(file)
        data["times"].append(y)


    with open(r"./instanceExamples/exec.json", "w") as file:
        json.dump(data, file)

    #cmd = ('docker-machine ssh manager docker stack deploy --compose-file updated-docker-compose.yml p ').split()
    cmd = ('docker-machine ssh '+str(Instance.nodes[0].name)+' docker stack deploy --compose-file updated-docker-compose.yml p ').split()
    #cmd = ('ssh root@'+str(Instance.nodes[0].name)+' docker stack deploy --compose-file updated-docker-compose.yml p ').split()
    

    p = subprocess.Popen(cmd)
    output, errors = p.communicate()


    # print(output)
    # print(errors)
#        ssh.connect(hostname='ec2-54-87-55-164.compute-1.amazonaws.com', username='ubuntu',pkey=privkey)
#        #ssh.connect("ubuntu@ec2-54-87-55-164.compute-1.amazonaws.com",)
#        stdin, stdout, stderr=ssh.exec_command('rmdir test')
#        print (stdout.read())
#        print(stderr.read())



    result=jsonify(time)

    return (result)
@app.route('/getexectime/', methods=['GET'])

def gettimes():
    f = open(r"./instanceExamples/exec.json", "r")

    data = json.load(f)
    response = app.response_class(
        response=json.dumps(data),
        mimetype='application/json'
    )
    return response

if __name__ == '__main__':
     app.run(port='5002')


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


1
2
4
6
7
8
['1', '2', '4', '6', '7', '8']

node 0 manager
node 1 worker1
node 2 worker2



current state : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


constraints : [[0, 1, 2, -1], [1, 2], [0, 1, 2, -1], [1, 2], [1, 2], [0, 1, 2, -1], [1, 2], [2], [0, 1, 2, -1], [0, 1, 2, -1], [0, 1, 2, -1], [1], [0, 1, 2, -1], [1, 2], [0, 1, 2, -1], [1, 2], [0, 1, 2, -1], [1, 2], [0, 1, 2, -1], [1], [1], [0, 1, 2, -1], [1, 2], [0, 1, 2, -1], [0, 1, 2, -1], [2], [2], [1], [0, 1, 2, -1], [2], [0, 1, 2, -1], [1], [0, 2], [0, 1, 2, -1], [0], [1], [2], [0, 1, 2, -1], [1], [1, 2], [2], [0, 1, 2, -1], [1]]


objectives : [{'1': 0.2}, {'2': 0.2}, {'4': 0.2}, {'6': 0.2}, {'7': 0.2}, {'8': 1}]


dependencies : [(16, 17)]



Algorithm running....


1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734

5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
0
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
0
5.141094526666734
2
5.141094526666734
0
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
1
5.141094526666734
210.7848755933361
[2, 1, 2, 2, 1, 1, 2, 2, 0, -1, 1, 1, 2, 1, 0, 1, 0, 1, -1, 1, 1, 1, 2, 2, 0, 2, 2, 1, 1, 2, 2, 1, 0, 2, 0, 1, 2, 1, 1, 1, 2, 1, 1]
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
0
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
0
5.141094526666734
1
5.141094526666734
0
5.141094526666734
1
5.141094526666734
1
5.14109452

5.141094526666734
0
5.141094526666734
0
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
0
5.141094526666734
1
5.141094526666734
364.6799999999999
Solution(variables=[-1, 1, 1, 1, 1, -1, 2, 2, 0, -1, 0, 1, 2, 1, -1, 2, 1, 2, -1, 1, 1, 1, 1, 1, -1, 2, 2, 1, 0, 2, 1, 1, 0, -1, 0, 1, 2, 2, 1, 2, 2, 0, 1],objectives=[100000, 100000, 100000, 100000, 100000],constraints=[0, 3, 0, 0, 0])
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
1
5.141094526666734
2
5.141094526666734
2
5.141094526666734
2
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141094526666734
1
5.141094526666734
2
5.141

...

127.0.0.1 - - [25/Jan/2022 12:19:30] "GET /getmem/ HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2022 12:26:41] "GET /getjson/ HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2022 12:26:42] "POST /geturl/ HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2022 12:26:42] "GET /getjson/ HTTP/1.1" 200 -
127.0.0.1 - - [25/Jan/2022 12:26:42] "GET /getexectime/ HTTP/1.1" 200 -


content http://192.168.99.115:9090/


[2022-01-25 12:26:42,399] ERROR in app: Exception on /getmem/ [GET]
Traceback (most recent call last):
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask_restful/__init__.py", line 271, in error_router
    return original_handler(e)
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/anwar/opt/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise 

[['ID', 'HOSTNAME', 'STATUS', 'AVAILABILITY', 'MANAGER', 'STATUS', 'ENGINE', 'VERSION'], ['wa96e2xslz4c6wc2qczpvkn17', 'manager', 'Ready', 'Active', 'Leader', '19.03.12'], ['wnblyovn3sspvv3k9siarxwek', 'worker1', 'Ready', 'Active', '19.03.12'], ['5n8981nq8sfm9jr1rdgr02oe3', 'worker2', 'Ready', 'Active', '19.03.12']]


127.0.0.1 - - [25/Jan/2022 12:30:19] "GET /getjson/ HTTP/1.1" 200 -
